###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)

# This notebook outlines the download and formatting process for the Center for Neighborhood Technology's Housing and Transportation Cost Index as well as the data combined from our end for counties and places in the GNRC operating region.  

Go to this page: https://htaindex.cnt.org/download/  
Upon registering for access, download the following documents:  
+ HTA Index for Counties in Tennessee and Kentucky  
+ HTA Index for MPOs  
+ HTA Index for Block Groups in Tennessee  

Save these csvs as they come in the Data Downloads folder of Parent Data Gathering  

### Calculations are made both for Comphrehensive plans at higher geography levels (counties, MPO), but also by block group to identify distressed areas at a granular level.

In [2]:
#Load API Key
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)
#create a variable that contains your api key
census_key = keys_dict_2['CENSUS']
bea_key = keys_dict_2['BEA']

In [3]:
GNRC = ['111', #Macon
       '161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149', #Rutherford
       '119'] #Maury
KY = ['003', #Allen
      '213'] #Simpson
slaces = ['02180', #Ashland City: Cheatham
          '39660', #Kingston Springs: Cheatham
          '57480', #Pegram: Cheatham
          '59560', #Pleasant View: Cheatham
          '04620', #Belle Meade: Davidson
          '05140', #Berry Hill: Davidson
          '27020', #Forest Hills: Davidson
          '29920', #Goodlettsville: Davidson/Sumner
          '52006', #Nashville-Davidson metropolitan government (balance): Davidson
          '54780', #Oak Hill: Davidson
          '63140', #Ridgetop: Davidson/Robertson
          '09880', #Burns: Dickson
          '13080', #Charlotte: Dickson
          '20620', #Dickson: Dickson
          '69080', #Slayden: Dickson
          '76860', #Vanleer: Dickson
          '79980', #White Bluff: Dickson
          '24320', #Erin: Houston
          '73460', #Tennessee Ridge: Houston/Stewart
          '44840', #McEwen: Humphreys
          '52820', #New Johnsonville: Humphreys
          '78560', #Waverly: Humphreys
          '16540', #Columbia: Maury
          '51080', #Mount Pleasant: Maury
          '70580', #Spring Hill: Maury/Williamson
          '15160', #Clarksville: Montgomery
          '00200', #Adams: Robertson
          '11980', #Cedar Hill: Robertson
          '16980', #Coopertown: Robertson
          '18420', #Cross Plains: Robertson
          '30960', #Greenbrier: Robertson
          '48980', #Millersville: Robertson/Sumner
          '60280', #Portland: Robertson/Sumner
          '70500', #Springfield: Robertson
          '80200', #White House: Robertson/Sumner
          '22360', #Eagleville: Rutherford
          '41200', #La Vergne: Rutherford
          '51560', #Murfreesboro: Rutherford
          '69420', #Smyrna: Rutherford
          '18820', #Cumberland City: Stewart
          '21400', #Dover: Stewart
          '28540', #Gallatin: Sumner
          '33280', #Hendersonville: Sumner
          '79420', #Westmoreland: Sumner
          '08280', #Brentwood: Williamson
          '25440', #Fairview: Williamson
          '27740', #Franklin: Williamson
          '53460', #Nolensville: Williamson
          '73900', #Thompson's Station: Williamson
          '41520', #Lebanon: Wilson
          '50780', #Mount Juliet: Wilson
          '78320', #Watertown: Wilson
          '40160', #Lafayette: Macon
          '69114', #Scottsville city, Kentucky
          '28918', #Franklin city, Kentucky
          '49460'] #Mitchelville city: Sumner

In [171]:
#2019 ACS 5 Year Median Household Income, total occupied housing units for households
#counties
url_str= 'https://api.census.gov/data/2019/acs/acs5?key='+census_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Median Household Income', 'Households', 'StateFIPS', 'GeoFIPS']
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)                                                              
col_names = ['NAME', 'GEO_ID', 'Median Household Income', 'Households', 'StateFIPS', 'GeoFIPS']
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = pd.concat([df, kycos], axis = 0)
#ky places call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Median Household Income', 'Households', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#places
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Median Household Income', 'Households', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#state call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Median Household Income', 'Households', 'StateFIPS']
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = pd.concat([df, state], axis = 0)
#national call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Median Household Income', 'Households', 'StateFIPS']
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = pd.concat([df, national], axis = 0)
predicates= {} #block groups GNRC Region
get_vars= ["NAME", 'B19013_001E', 'B25002_002E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "block group:*"
predicates["in"]= "state:47, county:*, tract:*"
data= requests.get(url_str, params = predicates)
col_names = ['NAME', 'Median Household Income', 'Households', 'StateFIPS', 'CountyFIPS', 'Census Tract', 'Block Group']
bg=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
bg['GEO_ID'] = bg['StateFIPS'] + bg['CountyFIPS'] + bg['Census Tract'] + bg['Block Group']
bg['GeoFIPS'] = bg['CountyFIPS'] + bg['Census Tract'] + bg['Block Group']
bg = bg.loc[bg['CountyFIPS'].isin(GNRC)]
bg = bg.drop(columns = ['CountyFIPS', 'Census Tract', 'Block Group']).reset_index(drop = True)
df = pd.concat([df, bg], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [172]:
hhincome = savename.reset_index(drop = True)
hhincome = hhincome.drop(columns = ['StateFIPS', 'GeoFIPS'])
hhincome['Median Household Income'] = hhincome['Median Household Income'].astype(float)
hhincome['Households'] = hhincome['Households'].astype(float)

In [173]:
hhincome['Annual Median Household Income'] = hhincome['Median Household Income']
hhincome['Monthly Median Household Income'] = hhincome['Median Household Income']/12
hhincome = hhincome.drop(columns = 'Median Household Income')

In [174]:
hhincome.tail()

,NAME,GEO_ID,Households,Annual Median Household Income,Monthly Median Household Income
1175,"Block Group 2, Census Tract 156.22, Davidson C...",470370156222,1340.0,86442.0,7203.500000
1176,"Block Group 2, Census Tract 190.06, Davidson C...",470370190062,676.0,45000.0,3750.000000
1177,"Block Group 1, Census Tract 202.09, Sumner Cou...",471650202091,732.0,63611.0,5300.916667
1178,"Block Group 2, Census Tract 108.02, Davidson C...",470370108022,366.0,40128.0,3344.000000
1179,"Block Group 4, Census Tract 188.01, Davidson C...",470370188014,334.0,94184.0,7848.666667


## H&T

In [175]:
kycos = pd.read_csv('../../Data Downloads/htaindex2019_data_counties_21.csv')
tncos = pd.read_csv('../../Data Downloads/htaindex2019_data_counties_47.csv')
#mpos = pd.read_csv('../../Data Downloads/htaindex2019_data_mpos_.csv')
tnplaces = pd.read_csv('../../Data Downloads/htaindex2019_data_places_47.csv')
kyplaces = pd.read_csv('../../Data Downloads/htaindex2019_data_places_21.csv')
tnbg = pd.read_csv('../../Data Downloads/htaindex2019_data_blkgrps_47.csv')

In [176]:
# mpos['name'] = mpos['name'].str.strip('\"')
# mpos['GEO_ID'] = mpos['mpo'].str.strip('\"')
# mpos = mpos.loc[mpos['name'] == 'Nashville Area MPO']

In [177]:
tncos['GEO_ID'] = tncos['county'].str.strip('\"')
kycos['GEO_ID'] = kycos['county'].str.strip('\"')
tnplaces['GEO_ID'] = tnplaces['place'].str.strip('\"')
kyplaces['GEO_ID'] = kyplaces['place'].str.strip('\"')
tnbg['GEO_ID'] = tnbg['blkgrp'].str.strip('\"')

In [178]:
tncos = tncos[['name', 'GEO_ID', 'h_cost', 't_cost_ami']]
kycos = kycos[['name', 'GEO_ID', 'h_cost', 't_cost_ami']]
tnplaces = tnplaces[['name', 'GEO_ID', 'h_cost', 't_cost_ami']]
kyplaces = kyplaces[['name', 'GEO_ID', 'h_cost', 't_cost_ami']]
tnbg = tnbg[['GEO_ID', 'h_cost', 't_cost_ami']]
#mpos = mpos[['name', 'GEO_ID', 'h_cost', 't_cost_ami']]

In [179]:
tncos['h_cost_ami'] = tncos['h_cost']*12
kycos['h_cost_ami'] = kycos['h_cost']*12
tnplaces['h_cost_ami'] = tnplaces['h_cost']*12
kyplaces['h_cost_ami'] = kyplaces['h_cost']*12
tnbg['h_cost_ami'] = tnbg['h_cost']*12
#mpos['h_cost_ami'] = mpos['h_cost']*12

In [180]:
tncos= tncos.drop(columns = 'h_cost')
kycos= kycos.drop(columns = 'h_cost')
tnplaces= tnplaces.drop(columns = 'h_cost')
kyplaces= kyplaces.drop(columns = 'h_cost')
tnbg= tnbg.drop(columns = 'h_cost')
#mpos= mpos.drop(columns = 'h_cost')

In [181]:
tncos['GEO_ID'] = tncos['GEO_ID'].str[2:]
kycos['GEO_ID'] = kycos['GEO_ID'].str[2:]
tnplaces['GEO_ID'] = tnplaces['GEO_ID'].str[2:]
kyplaces['GEO_ID'] = kyplaces['GEO_ID'].str[2:]
tnbg['CountyFIPS'] = tnbg['GEO_ID'].str[2:5]

In [182]:
tncos = tncos.loc[tncos['GEO_ID'].isin(GNRC)].reset_index(drop = True)
kycos = kycos.loc[kycos['GEO_ID'].isin(KY)].reset_index(drop = True)
tnplaces = tnplaces.loc[tnplaces['GEO_ID'].isin(slaces)].reset_index(drop = True)
kyplaces = kyplaces.loc[kyplaces['GEO_ID'].isin(slaces)].reset_index(drop = True)
tnbg = tnbg.loc[tnbg['CountyFIPS'].isin(GNRC)]
tnbg = tnbg.drop(columns = ['CountyFIPS']).reset_index(drop = True)

In [183]:
tncos['GEO_ID'] = '0500000US47' + tncos['GEO_ID']
kycos['GEO_ID'] = '0500000US21' + kycos['GEO_ID']
tnplaces['GEO_ID'] = '1600000US47' + tnplaces['GEO_ID']
kyplaces['GEO_ID'] = '1600000US21' + kyplaces['GEO_ID']

In [184]:
df = pd.concat([tncos, kycos, tnplaces, kyplaces, tnbg]).reset_index(drop = True)

In [185]:
data = df.merge(hhincome, on = 'GEO_ID')

In [186]:
data = data.drop(columns = 'name')

In [187]:
data = data.set_index(['NAME', 'GEO_ID'])

In [188]:
cols = list(data.columns)
data[cols] = data[cols].astype(float)

In [189]:
def percent(x, y):
    return (x/y)*100

In [190]:
data['Annual Median Household Income'] = data['Annual Median Household Income']
data['Monthly Median Household Income'] = data['Annual Median Household Income']/12
data['Annual Transportation Cost'] = data['t_cost_ami']
data['Monthly Transportation Cost'] = data['t_cost_ami']/12
data['Annual Housing Cost'] = data['h_cost_ami']
data['Monthly Housing Cost'] = data['h_cost_ami']/12
data['Annual Housing and Transportation Cost'] = data['Annual Housing Cost'] + data['Annual Transportation Cost']
data['Monthly Housing and Transportation Cost'] = (data['Annual Housing Cost'] + data['Annual Transportation Cost'])/12
data['Transportation Cost as Percent of Monthly Median Household Income'] = percent(data['Monthly Transportation Cost'], data['Monthly Median Household Income'])
data['Transportation Cost as Percent of Annual Median Household Income'] = percent(data['Annual Transportation Cost'], data['Annual Median Household Income'])
data['Housing Cost as Percent of Monthly Median Household Income'] = percent(data['Monthly Housing Cost'], data['Monthly Median Household Income'])
data['Housing Cost as Percent of Annual Median Household Income'] = percent(data['Annual Housing Cost'], data['Annual Median Household Income'])
data['Housing and Transportation Cost as Percent of Monthly Median Household Income'] = percent(data['Monthly Housing and Transportation Cost'], data['Monthly Median Household Income'])
data['Housing and Transportation Cost as Percent of Annual Median Household Income'] = percent(data['Annual Housing and Transportation Cost'], data['Annual Median Household Income'])

In [191]:
data = data.drop(columns = ['t_cost_ami', 'h_cost_ami'])

In [192]:
data = data.reset_index()

In [193]:
data.tail()

,NAME,GEO_ID,Households,Annual Median Household Income,Monthly Median Household Income,Annual Transportation Cost,Monthly Transportation Cost,Annual Housing Cost,Monthly Housing Cost,Annual Housing and Transportation Cost,Monthly Housing and Transportation Cost,Transportation Cost as Percent of Monthly Median Household Income,Transportation Cost as Percent of Annual Median Household Income,Housing Cost as Percent of Monthly Median Household Income,Housing Cost as Percent of Annual Median Household Income,Housing and Transportation Cost as Percent of Monthly Median Household Income,Housing and Transportation Cost as Percent of Annual Median Household Income
1172,"Block Group 2, Census Tract 1203, Houston Coun...",470831203002,521.0,37697.0,3141.416667,14868.0,1239.000000,11280.0,940.0,26148.0,2179.000000,39.440804,39.440804,29.922806,29.922806,69.363610,69.363610
1173,"Block Group 1, Census Tract 1202, Houston Coun...",470831202001,454.0,43125.0,3593.750000,14825.0,1235.416667,9612.0,801.0,24437.0,2036.416667,34.376812,34.376812,22.288696,22.288696,56.665507,56.665507
1174,"Block Group 2, Census Tract 1304, Humphreys Co...",470851304002,363.0,42240.0,3520.000000,15846.0,1320.500000,10296.0,858.0,26142.0,2178.500000,37.514205,37.514205,24.375000,24.375000,61.889205,61.889205
1175,"Block Group 3, Census Tract 1203, Houston Coun...",470831203003,238.0,30700.0,2558.333333,14708.0,1225.666667,9324.0,777.0,24032.0,2002.666667,47.908795,47.908795,30.371336,30.371336,78.280130,78.280130
1176,"Block Group 2, Census Tract 1202, Houston Coun...",470831202002,340.0,45104.0,3758.666667,14876.0,1239.666667,10440.0,870.0,25316.0,2109.666667,32.981554,32.981554,23.146506,23.146506,56.128060,56.128060


In [194]:
data.to_csv('../../Outputs/CNT_HT.csv')